# MVPA Tutorial

In this tutorial, you will be using python and a few packages to perform MVPA analysis. This is a difficult type of analysis that takes a long time to master, so we'll start out with showing you some basic operations. 
Please don't let the programming code scare you, but try to focus on the figures that are produced by the analyses. 

In [ ]:
# python is a lean programming language, 
# which means that we have to explicitly import specialized functionality
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import nilearn, sklearn

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


# How does fitting work?

This example demonstrates the problems of underfitting and overfitting. The plot shows the function that we want to approximate, which is a part of the cosine function. In addition, the (noisy) samples from the real function and the approximations of different models are displayed. The models have polynomial features of different degrees, the more degrees, the more parameters. If your model is too simple, it will not describe the data well. We can see that a linear function (polynomial with degree 1) is not sufficient to fit the training samples. This is called underfitting. 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation

np.random.seed(0)

n_samples = 50
degrees = [1, 2, 4, 8]

true_fun = lambda X: np.cos(1.5 * np.pi * X)
X = np.sort(np.random.rand(n_samples))
y = true_fun(X) + np.random.randn(n_samples) * 0.1

plt.figure(figsize=(12, 5))
ax = plt.subplot(111)
plt.setp(ax, xticks=(), yticks=())

polynomial_features = PolynomialFeatures(degree=1,
                                         include_bias=False)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),
                     ("linear_regression", linear_regression)])
pipeline.fit(X[:, np.newaxis], y)

# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline,
    X[:, np.newaxis], y, scoring="mean_squared_error", cv=10)

X_test = np.linspace(0, 1, 100)
plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model, degree 1: Too Simple")
plt.plot(X_test, true_fun(X_test), label="True function")
plt.scatter(X, y, label="Samples")
plt.xlabel("x")
plt.ylabel("y")
plt.xlim((0, 1))
plt.ylim((-2, 2))
plt.legend(loc="best")
plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(
    degrees[0], -scores.mean(), scores.std()));

### Making our model more complex.
If we have more data points, we can add complexity to our model, but not too much! Here, we're using different model complexities, which illustrate how important it is that you don't fit a model that is overly complex (overfitting), or overly simple (underfitting). 

In [ ]:
plt.figure(figsize=(20, 5))
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)
    plt.setp(ax, xticks=(), yticks=())

    polynomial_features = PolynomialFeatures(degree=degrees[i],
                                             include_bias=False)
    linear_regression = LinearRegression()
    pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
    pipeline.fit(X[:, np.newaxis], y)

    # Evaluate the models using crossvalidation
    scores = cross_validation.cross_val_score(pipeline,
        X[:, np.newaxis], y, scoring="mean_squared_error", cv=10)

    X_test = np.linspace(0, 1, 100)
    plt.plot(X_test, pipeline.predict(X_test[:, np.newaxis]), label="Model")
    plt.plot(X_test, true_fun(X_test), label="True function")
    plt.scatter(X, y, label="Samples")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim((0, 1))
    plt.ylim((-2, 2))
    plt.legend(loc="best")
    plt.title("Degree {}\nMSE = {:.2e}(+/- {:.2e})".format(
        degrees[i], -scores.mean(), scores.std()))

You will see, for high degrees the model will overfit the training data, i.e. it learns the noise that is specific to the training data. Imagine you were going to use an independent dataset to test whether the classification algorithm has done a good job. Which would you choose?

*Further explanation:*

Our cross-validation shows that a polynomial of degree 4 approximates the true function almost perfectly.  Technically, in this example we evaluate quantitatively whether we are overfitting / underfitting by using cross-validation.  As a measure of fit quality, we calculate the mean squared error (MSE) on a validation set, the higher, the less likely the model generalizes correctly from the training data.

# Comparing Classifiers

### Difference between different classifiers

In this first example, we're going to explore some differences between different classifiers. 
One big difference between classifiers is whether they are 'linear' or not. You can think of linear in this context as being constrained to very simple straight (*linear*) discriminating lines. 

Machine Learning is a very cool hot field in data analysis, being used by the likes of Google and Facebook. If you are interested in the mechanisms behind the different methods, I recomment the Coursera course Machine Learning, from Stanford https://www.coursera.org/learn/machine-learning. For now, we won't dive into the specifics of these methods lest we get too mathematical. Take home message here is that depending on the characteristics of a given problem, one will normally choose a method that suits these characteristics to optimize the performance of the classifier.  

In [ ]:
# all the specific classifiers that we'll use in the first example.
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

Now, we create the classifiers and some data to classify. 
Since we generate the data ourselves, we can play with the amount of noise in the classification problem, making it easier and/or harder to perform. The amount of noise is set in the last lines of the cell below. Once you have run this section up until the plots are produced, you can try running this section again with a higher amount of noise. 

In [ ]:
# create classifiers and data. 
h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear Support Vector Machine\n(SVM)", "Radial Basis Function SVM", "Decision Tree",
         "Random Forest", "AdaBoost", "Naive Bayes", "Linear Discriminant Analysis",
         "Quadratic Discriminant Analysis"]
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [linearly_separable,
            make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1)
            ]

First, we describe our data by plotting them. Looking at your data is always extremely important, it's the only way to 'see' what's really going on! Training points are plotted as circles, test points are plotted as triangles. The two classes are shown in different colors, red and blue. 

In [ ]:
figure = plt.figure(figsize=(12, 5))
i = 1
# iterate over datasets
for ds in datasets:
    ax = plt.subplot(1, len(datasets), i)
    # preprocess dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright, s = 60)
    # and testing points
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.3, s = 60, marker = 'v')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title('Dataset %i'%i)
    i += 1

figure.subplots_adjust(left=.02, right=.98)

Do you see the patterns in the data? Can you think of a simple rule (like drawing a line between red and blue) that would distinguish between the two colors, or classes? Which classification problem do you think is easiest?

In [ ]:
figure = plt.figure(figsize=(27, 9))
i = 1
# iterate over datasets
for ds in datasets:
    # preprocess dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright)
    # and testing points
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, m_max]x[y_min, y_max].
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plot also the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright)
        # and testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                   alpha=0.6)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=15, horizontalalignment='right')
        i += 1

figure.subplots_adjust(left=.02, right=.98)

The decision boundaries for each of the classification algorithms are shown in the underlying colors, from red to blue. Some algorithms, by virtue of their architecture, will only draw single straight decision boundaries, others can draw multiple straight decision boundaries, and yet others can draw curved decision boundaries. 

**Try to answer the following questions:**
1. How do the linear versions of *Discriminant analysis* and *Support Vector Machines (SVM)* go wrong? 
2. What is the difference between classification problem 1 and 2&3, which makes it more amenable to classification by the linear SVM and Discriminant analysis? 
3. Try to think of a downside of using a classifier that can draw very complex decision boudaries. 

Answers:

1.

2.

3.


# NILearn: neuroimaging analysis in a notebook  

NILearn [http://nilearn.github.io] is a package that allows us to visualize and analyze neuroimaging data. It performs Machine Learning type analyses from sklearn (which we just used the classifiers from) on brain imaging data easily. 

First, an example on how we would plot neuroimaging data using this package. We will plot the masks we will use for a following analysis. What this means is that we'll take data from these specific regions to train our MVPA algorithm. 

In [ ]:
from nilearn import datasets
from nilearn import plotting

haxby_dataset = datasets.fetch_haxby(n_subjects=1)
func_filename = haxby_dataset.func[0]

plotting.plot_glass_brain(haxby_dataset['mask_face'][0], title = 'Face-selective Regions');
plotting.plot_glass_brain(haxby_dataset['mask_house'][0], title = 'House-selective Regions');

And, an example of statistics of a simple task plotted in a glass brain. You can play around with the threshold values in the last lines of the following cell. 

In [ ]:
localizer_dataset = datasets.fetch_localizer_contrasts(
    contrasts = ['auditory&visual motor vs cognitive processing', 'checkerboard'], 
    n_subjects = 1, get_tmaps = True)

plotting.plot_glass_brain(localizer_dataset.cmaps[1], threshold=1.5, colorbar=True,
                          plot_abs=False, title = 'Perception vs Cognition', display_mode='ortho');
plotting.plot_glass_brain(localizer_dataset.tmaps[1], threshold=1, colorbar=True,
                          plot_abs=False, title = 'High-contrast visual stimulus', display_mode='ortho');

# ROI-based decoding analysis in Haxby et al. dataset

Here, we reproduce the data analysis conducted by Haxby et al. in *“Distributed and Overlapping Representations of Faces and Objects in Ventral Temporal Cortex”*, the first Science paper that started the field of MVPA.

#### The Haxby 2001 experiment
Subjects are presented visual stimuli from different categories. We are going to predict which category the subject is seeing from the fMRI activity recorded in masks of the ventral stream. Significant prediction shows that the signal in the region contains information on the corresponding category.

Specifically, we look at decoding accuracy for different objects in three different masks: the full ventral stream (**mask_vt**), the house selective areas (**mask_house**) and the face selective areas (**mask_face**), that have been defined via a standard GLM-based analysis.

First we load and prepare the data

In [ ]:
# Load nilearn NiftiMasker, the practical masking and unmasking tool
from nilearn.input_data import NiftiMasker

# load labels
import numpy as np
labels = np.recfromcsv(haxby_dataset.session_target[0], delimiter=" ")
stimuli = labels['labels']

# identify resting state labels in order to be able to remove them
resting_state = stimuli == b"rest"

# find names of remaining active labels
categories = np.unique(stimuli[np.logical_not(resting_state)])

# extract tags indicating to which acquisition run a tag belongs
session_labels = labels["chunks"][np.logical_not(resting_state)]

Then we use scikit-learn for decoding on the different masks. With all the different stimulus categories this may take a while. The program will keep you posted on its results. 

In [ ]:
# The classifier: a support vector classifier
from sklearn.svm import SVC
classifier = SVC(C=1., kernel="linear")

# A classifier to set the chance level
from sklearn.dummy import DummyClassifier
dummy_classifier = DummyClassifier()

# Make a data splitting object for cross validation
from sklearn.cross_validation import LeaveOneLabelOut, cross_val_score
cv = LeaveOneLabelOut(session_labels)

mask_names = ['mask_vt', 'mask_face', 'mask_house']

mask_scores = {}
mask_chance_scores = {}

for mask_name in mask_names:
    print("Working on mask %s" % mask_name)
    # For decoding, standardizing is often very important
    mask_filename = haxby_dataset[mask_name][0]
    masker = NiftiMasker(mask_img=mask_filename, standardize=True)
    masked_timecourses = masker.fit_transform(
        func_filename)[np.logical_not(resting_state)]

    mask_scores[mask_name] = {}
    mask_chance_scores[mask_name] = {}

    for category in categories:
        print("Processing %s %s" % (mask_name, category))
        task_mask = np.logical_not(resting_state)
        classification_target = (stimuli[task_mask] == category)
        mask_scores[mask_name][category] = cross_val_score(
            classifier,
            masked_timecourses,
            classification_target,
            cv=cv, scoring="f1")

        mask_chance_scores[mask_name][category] = cross_val_score(
            dummy_classifier,
            masked_timecourses,
            classification_target,
            cv=cv, scoring="f1")

        print("Scores: %1.2f +- %1.2f" % (
            mask_scores[mask_name][category].mean(),
            mask_scores[mask_name][category].std()))


We make a simple bar plot to summarize the results. Can you interpret these results?

For example, where (in what mask) is the classification accuracy for houses best? And in what region can you best decode faces?

In [ ]:
plt.figure()

tick_position = np.arange(len(categories))
plt.xticks(tick_position, categories, rotation=45)

for color, mask_name in zip('rgb', mask_names):
    score_means = [mask_scores[mask_name][category].mean()
                   for category in categories]
    plt.bar(tick_position, score_means, label=mask_name,
            width=.25, color=color)

    score_chance = [mask_chance_scores[mask_name][category].mean()
                    for category in categories]
    plt.bar(tick_position, score_chance,
            width=.25, edgecolor='k', facecolor='none')

    tick_position = tick_position + .2

plt.ylabel('Classification accurancy (f1 score)')
plt.xlabel('Visual stimuli category')
plt.legend(loc='best')
plt.title('Category-specific classification accuracy for different masks')
plt.tight_layout()


# Searchlight decoding analysis

Searchlight was introduced in Information-based functional brain mapping, Nikolaus Kriegeskorte, Rainer Goebel and Peter Bandettini (PNAS 2006) and consists of scanning the images volume with a searchlight. Briefly, a ball of given radius is scanned across the brain volume and the prediction accuracy of a classifier trained on the corresponding voxels is measured.

For simplicity, we are going to decode from exactly the same experiment as the ROI-based analysis above. So, it's just houses, faces and objects that we're trying to distinguish from one another. 

First we download some data. 

In [ ]:
from nilearn.image import new_img_like
from nilearn.image import index_img
from nibabel import load

haxby_dataset = datasets.fetch_haxby_simple()

fmri_filename = haxby_dataset.func[0]
y, session = np.loadtxt(haxby_dataset.session_target[0]).astype('int').T
conditions = np.recfromtxt(haxby_dataset.conditions_target[0])['f0']

We'll set things up and fit the Searchlight decoder on a single slice of data (otherwise it would take too much time). Keep in mind that this may take up to a minute to process. If you see 'error' messages popping up, don't worry too much. Just try the next cell. 

In [ ]:
condition_mask = np.logical_or(conditions == b'face', conditions == b'house')

fmri_img = index_img(fmri_filename, condition_mask)
y, session = y[condition_mask], session[condition_mask]
conditions = conditions[condition_mask]

mask_img = load(haxby_dataset.mask)

# .astype() makes a copy.
process_mask = mask_img.get_data().astype(np.int)
picked_slice = 27
process_mask[..., (picked_slice + 1):] = 0
process_mask[..., :picked_slice] = 0
process_mask[:, 30:] = 0
process_mask_img = new_img_like(mask_img, process_mask)

# Make processing parallel
# /!\ As each thread will print its progress, n_jobs > 1 could mess up the
#     information output.
n_jobs = 1

# Define the cross-validation scheme used for validation.
# Here we use a KFold cross-validation on the session, which corresponds to
# splitting the samples in 4 folds and make 4 runs using each fold as a test
# set once and the others as learning sets
from sklearn.cross_validation import KFold
cv = KFold(y.size, n_folds=4)

import nilearn.decoding
# The radius is the one of the Searchlight sphere that will scan the volume
searchlight = nilearn.decoding.SearchLight(
    mask_img,
    process_mask_img=process_mask_img,
    radius=5.6, n_jobs=n_jobs,
    verbose=1, cv=cv)
searchlight.fit(fmri_img, y)

Now, we have to prepare the results for plotting, and perform the actual plotting. This time, we're not using a glass brain, but the background image is our actual functional data. Looks less nice...

In [ ]:
from nilearn.input_data import NiftiMasker

# For decoding, standardizing is often very important
nifti_masker = NiftiMasker(mask_img=mask_img, sessions=session,
                           standardize=True, memory='nilearn_cache',
                           memory_level=1)
fmri_masked = nifti_masker.fit_transform(fmri_img)

from sklearn.feature_selection import f_classif
f_values, p_values = f_classif(fmri_masked, y)
p_values = -np.log10(p_values)
p_values[p_values > 10] = 10
p_unmasked = nifti_masker.inverse_transform(p_values).get_data()

# Use the fmri mean image as a surrogate of anatomical data
from nilearn import image
mean_fmri = image.mean_img(fmri_img)

from nilearn.plotting import plot_stat_map, show
plotting.plot_stat_map(new_img_like(mean_fmri, searchlight.scores_), mean_fmri,
              title="Searchlight", cut_coords = [-4, -65, -16],
              colorbar=False)

# F_score results
p_ma = np.ma.array(p_unmasked, mask=np.logical_not(process_mask))
plotting.plot_stat_map(new_img_like(mean_fmri, p_ma), mean_fmri,
              title="F-scores", cut_coords = [-4, -65, -16],
              colorbar=False)
